In [43]:
import requests
import json
from bs4 import BeautifulSoup

def get_page(url):
    req = requests.get(url)
    soup = BeautifulSoup(req.text, "lxml")
    json_script = soup.find("script", type="application/ld+json")

    # If we got the content
    if json_script is not None:
        script_content = json.loads(json_script.text)
        # If the last thing in the navigation is "Cocktail Recipes"
        if ("Cocktail Recipes" == script_content[0]['itemListElement'][-1]['item']['name']):
            script_content = json.loads(json_script.text)
            drink_name = script_content[1]['name']
            ingredients = script_content[1]["recipeIngredient"]
            instr = script_content[1]["recipeInstructions"]

            step_list = []
            for step in instr:
                step_list.append(step["text"])

            with open("martha_stewart.json", 'a') as outfile:
                data = {"name": drink_name, "ingredients": ingredients, "instructions": step_list}


In [46]:
get_page("https://www.marthastewart.com/1165530/french-martini")

French Martini
['1 1/2 ounces vodka', '1/2 ounce Chambord', '3/4 ounces pineapple juice', 'Lemon twist, for garnish']
Fill a cocktail shaker halfway with ice. Add in vodka, Chambord, and pineapple juice. Shake well. Strain into a 4-ounce martini glass and garnish with lemon twist.


In [42]:
req = requests.get("https://www.marthastewart.com/314342/extra-dirty-martini")
soup = BeautifulSoup(req.text, "lxml")
json_script = soup.find("script", type="application/ld+json")
script_content = json.loads(json_script.text)
print("Cocktail Recipes" == script_content[0]['itemListElement'][-1]['item']['name'])

True


In [ ]:
# Esquire has somewhat semantic markup for recipes
# Ingredients are in a ingredients-body
# Instructions are in a directions-body
# Can probably filter URLs by having "drinks" in them, like
# <link href="https://www.esquire.com/food-drink/drinks/recipes/a3683/negroni-drink-recipe/" rel="canonical">

    

In [ ]:
# https://www.thecocktaildb.com/api.php
# Probably has some sort of rate limiting or something.
# May end up being an excuse to write a proxy spinner


In [ ]:
#https://makemeacocktail.com/recipes/Gin-cocktails/?ing=61
#https://kindredcocktails.com/cocktail/bananarac
#https://www.thecocktailproject.com/search-recipes
# URLS of the form https://www.diffordsguide.com/cocktails/recipe
#https://www.cocktailcontessa.com
#https://www.allrecipes.com/recipes/133/drinks/cocktails/
#https://www.seriouseats.com/drink-recipes-5117861
#https://imbibemagazine.com/category/recipes/cocktails-spirits-recipes/
#https://mrbostondrinks.com/recipes/blue-lagoon

In [52]:
url = "https://www.pinterest.com/pin/create/link/?url=https://www.marthastewart.com/2126005/halloween-cake-recipes?slide=094fbdda-1f06-42ae-8722-fe180a2413fd%3Futm_source=pinterest.com%26utm_medium=social%26utm_campaign=social-share-gallery%26utm_content=20210921&media=https%3A%2F%2Fimagesvc.meredithcorp.io%2Fv3%2Fmm%2Fimage%3Furl%3Dhttps%253A%252F%252Fassets.marthastewart.com%252Fstyles%252Fwmax-750%252Fd33%252Fcrawly-cake-phobias-1011mld107647%252Fcrawly-cake-phobias-1011mld107647_vert_0_horiz.jpg%253Fitok%253Df2RnN138&description=Creepy-Crawly%20Cake%20"
import re
matcher = re.compile("(http|https)://www\.marthastewart\.com")
print(re.match(matcher, url))
url = "https://www.marthastewart.com/2138473/pastry-chef-dominique-ansel-shares-lessons-learned-martha-stewart"
print(re.match(matcher, url))


None
<_sre.SRE_Match object; span=(0, 29), match='https://www.marthastewart.com'>


In [2]:
import requests
from bs4 import BeautifulSoup
req = requests.get("https://mrbostondrinks.com/recipes/blue-lagoon-2000")
soup = BeautifulSoup(req.text, "lxml")


In [3]:
req.text

'<!doctype html>\n<html class="no-js" ng-app="mrBostonRecipes" lang="en" ng-strict-di>\n  <head>\n    <!-- Google Tag Manager -->\n    <script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({\'gtm.start\':\n    new Date().getTime(),event:\'gtm.js\'});var f=d.getElementsByTagName(s)[0],\n    j=d.createElement(s),dl=l!=\'dataLayer\'?\'&l=\'+l:\'\';j.async=true;j.src=\n    \'https://www.googletagmanager.com/gtm.js?id=\'+i+dl;f.parentNode.insertBefore(j,f);\n    })(window,document,\'script\',\'dataLayer\',\'GTM-NSKCGD4\');</script>\n    <!-- End Google Tag Manager -->\n    <meta charset="utf-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1, user-scalable=no">\n    <title ng-bind-template="{{viewTitle}} | Mr. Boston Drinks"></title>\n    <base href="/index.html">\n    <meta property="og:site_name" content="Mr. Boston Drinks"/>\n    <meta name="fragment" content="!">\n    <meta property="fb:app_id" content="948323305227233" />\n    <meta name="p:domain_verify" conten

In [ ]:
from bs4 import BeautifulSoup
with open("./data/index.html", 'r') as infile:
    html = infile.read()
    bs = BeautifulSoup(html)
    print(bs.find("h1", attrs={"class": "recipe-name"}).text)

In [ ]:
import requests
from bs4 import BeautifulSoup
# Request to their backend that the "Load more" button does
responses = {}
for ii in range(30):
    req_data = {"context":"recipes","number":12, "value":0,"post_type":"recipe","page":ii}
    responses[ii] = requests.post("https://www.themixer.com/en-us/wp-json/site/v1/load-more", data=req_data)



In [ ]:
# Clean up the results
import json
import re
recipe_url_matcher = re.compile("https://www\.themixer\.com/en-us/recipes/[^/]*/")

recipes = []
for r in responses.values():
    data = json.loads(r.text)
    as_html = data["html"]
    urls = (re.findall(recipe_url_matcher, as_html))
    if len(urls) > 0:
        recipes.extend(urls)

In [ ]:
# Load up some random page
test_url = recipes[20]
page = requests.get(test_url)

In [30]:
bs = BeautifulSoup(page.text)

In [54]:
def mixer_to_dict(bs):
    name = bs.find("h1", attrs={"class", "title"}).text.strip()
    
    ingred_list = []
    ingredients = bs.find_all("div", attrs={"class": "d-flex align-items-top justify-content-between export-ingredient-item"})
    for ing in ingredients:
        ingredient = {}
        quant = ing.find("span", attrs = {"class":"unity_quantity"})
        if quant is not None:
            ingredient["amount"] = quant.text.strip()
        unit = ing.find("span", attrs = {"class":"unity_value"})
        if unit is not None:
            ingredient["unit"] = unit.text.strip()
        content = ing.find("div", attrs={"class": "content"})
        if content is not None:
            ingredient["name"] = content.text.strip()

        ingred_list.append(ingredient)

    instr_list = []
    instructions = bs.find_all("div", attrs={"class", "content fst-italic export-instructions-item"})
    for i in instructions:
        instr_list.append(i.text.replace("\n", ""))

    cocktail = {"name":name, "ingredients":ingred_list, "instructions":instr_list}
    return cocktail

In [55]:
import random # Don't go as hard as possible
import time

mixer_coctails = []
for url in recipes:
    page = requests.get(url)
    soup = BeautifulSoup(page.text)
    cocktail = mixer_to_dict(soup)
    mixer_coctails.append(cocktail)
    #time.sleep(random.random() * 2)



In [56]:
with open("./data/themixer_drinks.json", 'w') as outfile:
    json.dump(mixer_coctails, outfile)

In [ ]:
# Now let's go pillage Cocktail-society.com, starting with iterating all the index pages and then all the sub-pages on them
liquors = ["brandy", "gin", "mezcal", "rum", "tequila", "vodka", "whiskey"] 
# "other" and "shots" don't conform to the pattern, get them next
recipes = set()

def get_recipes(url):
    page = requests.get(url)
    bs = BeautifulSoup(page.text)
    links = bs.find_all("a", attrs={"class":"ct-link"})
    for link in links:
        if link['href'].startswith("https://cocktail-society.com/recipes/"):
            recipes.add(link['href'])
    next_page = bs.find("a", attrs={"class": "next page-numbers"})
    if next_page is not None:
        # LOL recursion machine go brrrrrrr
        get_recipes(next_page['href'])

for liquor in liquors:
    get_recipes(f"https://cocktail-society.com/category/recipes/{liquor}-cocktails/")
    


In [64]:
print(recipes)
len(recipes)

{'https://cocktail-society.com/recipes/when-the-smoke-cleared/', 'https://cocktail-society.com/recipes/gin-cocktails/ramos-gin-fizz/', 'https://cocktail-society.com/recipes/chocolate-martini/', 'https://cocktail-society.com/recipes/long-island-iced-tea/', 'https://cocktail-society.com/recipes/mezcal-dante-cocktail/', 'https://cocktail-society.com/recipes/glasgow-mule/', 'https://cocktail-society.com/recipes/manhattan-vs-old-fashioned-cocktail/', 'https://cocktail-society.com/recipes/christmas-mezcal-cocktail/', 'https://cocktail-society.com/recipes/jungle-bird/', 'https://cocktail-society.com/recipes/jack-frost/', 'https://cocktail-society.com/recipes/what-is-grog/', 'https://cocktail-society.com/recipes/bourbon-eggnog/', 'https://cocktail-society.com/recipes/singapore-sling-cocktail/', 'https://cocktail-society.com/recipes/classic-gin-gimlet-cocktail/', 'https://cocktail-society.com/recipes/succulent-blood-cocktail/', 'https://cocktail-society.com/recipes/sea-breeze/', 'https://cockta

246

In [73]:
url = "https://cocktail-society.com/recipes/devils-soul-cocktail/"
page= requests.get(url)

In [90]:
# There's a script tag that is basically a JSON representation of some of the content of the page, and seems to
# include the recipe itself. Going to try pulling the parts I want out of that and dumping that to JSON instead 
# of HTML parsing the soup. 
society_cocktails = []
for url in recipes:
    page = requests.get(url)
    bs = BeautifulSoup(page.text)
    json_tag = bs.find("script", class_="yoast-schema-graph")
    if json_tag is None:
        print(f"No JSON for {url}")
        continue
    data = json.loads(json_tag.text)

    cocktail = {}
    for entry in data["@graph"]:
        if entry["@type"] == "NewsArticle":
            cocktail["name"] = entry["headline"]
        if entry["@type"] == "Recipe":
            cocktail["ingredients"] = entry["recipeIngredient"]
            cocktail["instructions"] = []
            for instr in entry["recipeInstructions"]:
                cocktail["instructions"].append(instr["name"])

    society_cocktails.append(cocktail)
    time.sleep(random.random() * 3)

with open("./data/cocktail_society.json", 'w') as outfile:
    json.dump(society_cocktails, outfile, indent=4)